In [1]:
from pathlib import Path

import evaluate
from transformers import IntervalStrategy

from utils.dataset_utils import get_datasets, take_even_class_distribution_sample, \
    transform_for_inference
from utils.model_utils import POLITICAL_LEANING_NO_CENTER_LABEL_MAPPING, \
    POLITICAL_LEANING_WITH_CENTER_LABEL_MAPPING, finetune_custom_models

In [2]:
TRAIN_DATASET_SAMPLE_SIZE = 1_000
EVAL_DATASET_SAMPLE_SIZE = 100

train_datasets = []
eval_datasets = []
for dataset in get_datasets():
    label_mapping = POLITICAL_LEANING_WITH_CENTER_LABEL_MAPPING if dataset.has_center_leaning_class \
        else POLITICAL_LEANING_NO_CENTER_LABEL_MAPPING

    eval_dataset = take_even_class_distribution_sample(dataset, EVAL_DATASET_SAMPLE_SIZE)
    eval_dataset = transform_for_inference(eval_dataset, label_mapping)

    # Remove the eval sample from the source dataframe.
    dataset.dataframe = dataset.dataframe.loc[~dataset.dataframe.index.isin(eval_dataset.dataframe.index)]

    train_dataset = take_even_class_distribution_sample(dataset, TRAIN_DATASET_SAMPLE_SIZE)
    train_dataset = transform_for_inference(train_dataset, label_mapping)

    train_datasets.append(train_dataset.to_huggingface())
    eval_datasets.append(eval_dataset.to_huggingface())

In [3]:
TRAINING_SEED = 37
DATA_SEED = 37
METRIC = evaluate.load("accuracy")
EVAL_STRATEGY = IntervalStrategy.EPOCH

finetune_custom_models(
    Path("dataset_benchmark", "leave_one_in"),
    train_datasets,
    eval_datasets,
    METRIC,
    EVAL_STRATEGY,
    TRAINING_SEED,
    DATA_SEED
)

fine-tuning launch/POLITICS into dataset_benchmark/leave_one_in:
  article_bias_prediction


Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at launch/POLITICS and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at launch/POLITICS and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.884622,1.000000
2,No log,0.841997,1.000000
3,No log,0.843494,1.000000


  commoncrawl_news_articles


Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at launch/POLITICS and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at launch/POLITICS and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


KeyboardInterrupt: 